# 🧹 Cleaning Eye-Tracking Data

This notebook loads, filters, and saves cleaned eye-tracking data for further analysis.

In [12]:
# Import packages
import pandas as pd

# Load raw data
df = pd.read_csv("../data/raw/eye_tracking_2025.csv")
df.head()
print(f"All trials: {len(df)}")

All trials: 1679


In [13]:
# Drop test trails, where any of the key fields are 'UNDEFINED'
undefined_mask = (
    (df['Session_Name_'] == 'UNDEFINED') |
    (df['Trial_Index_'] == 'UNDEFINED') |
    (df['result'] == 'UNDEFINED') |
    (df['trialid'] == 'UNDEFINED')
)

df = df[~undefined_mask]

print(f"✅ Remaining rows after dropping UNDEFINED trials: {len(df)}")


✅ Remaining rows after dropping UNDEFINED trials: 1582
